# Use custom software_spec to create statsmodels function describing data with `ibm-watsonx-ai`

This notebook demonstrates how to deploy in watsonx.ai Runtime service as Python function with `statsmodel`, which requires creation of custom software specification using `requirements.txt` file with all required libraries.

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

-  Working with the watsonx.ai instance
-  Creating custom software specification
-  Online deployment of python function
-  Scoring data using deployed function

## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Function creation](#create)
3. [Function upload](#upload) 
4. [Web service creation](#deploy)
5. [Scoring](#score)
6. [Clean up](#cleanup)
7. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install wget | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1
%pip install statsmodels | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [ ]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate WML services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="create"></a>
## 2. Create function

In this section you will learn how to create deployable function with statsmodels module calculating describition of a given data.

#### Create deployable callable which uses `statsmodels` library

In [7]:
def deployable_callable():
    """
    Deployable python function with score
    function implemented.
    """

    try:
        from statsmodels.stats.descriptivestats import describe
    except ModuleNotFoundError as e:
        print(f"statsmodels not installed: {str(e)}")

    def score(payload):
        """
        Score method.
        """
        try:
            data = payload["input_data"][0]["values"]
            return {"predictions": [{"values": str(describe(data))}]}
        except Exception as e:
            return {"predictions": [{"values": [repr(e)]}]}

    return score

####  Test callable locally

In [8]:
import numpy as np

score_function = deployable_callable()

data = np.random.randn(10, 10)
data_description = score_function({"input_data": [{"values": data}]})

print(data_description["predictions"][0]["values"])

                          0          1          2          3          4  \
nobs              10.000000  10.000000  10.000000  10.000000  10.000000   
missing            0.000000   0.000000   0.000000   0.000000   0.000000   
mean               0.109419  -0.396759   0.230355  -0.300086  -0.404532   
std_err            0.366493   0.353975   0.292359   0.310328   0.303524   
upper_ci           0.827732   0.297020   0.803369   0.308147   0.190363   
lower_ci          -0.608894  -1.090538  -0.342659  -0.908318  -0.999428   
std                1.158952   1.119368   0.924521   0.981344   0.959826   
iqr                1.658504   0.502628   0.783272   1.306470   1.403269   
iqr_normal         1.229451   0.372599   0.580640   0.968488   1.040245   
mad                0.914476   0.643780   0.667696   0.786667   0.797205   
mad_normal         1.146126   0.806858   0.836833   0.985941   0.999148   
coef_var          10.591883  -2.821279   4.013468  -3.270214  -2.372679   
range              3.4888

<a id="upload"></a>
## 3. Upload python function

In this section you will learn how to upload the Python function to watsonx.ai.

#### Custom software_specification
Create new software specification based on `runtime-25.1-py3.12`.

In [9]:
requirements_txt_content = "statsmodels==0.14.4"

with open("requirements.txt", "w") as file:
    file.write(requirements_txt_content)

In [10]:
base_sw_spec_id = client.software_specifications.get_id_by_name("runtime-25.1-py3.12")

The `requirements.txt` file describes details of package extension. Now you need to store new package extension using `APIClient`.

In [11]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "statsmodels env",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with statsmodels",
    client.package_extensions.ConfigurationMetaNames.TYPE: "requirements_txt",
}

pkg_extn_details = client.package_extensions.store(
    meta_props=meta_prop_pkg_extn, file_path="requirements.txt"
)
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Creating package extensions
SUCCESS


#### Create new software specification and add created package extention to it.

In [12]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "statsmodels software_spec",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for statsmodels",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        "guid": base_sw_spec_id
    },
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)

client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)

SUCCESS


'SUCCESS'

#### Get the details of created software specification

In [13]:
client.software_specifications.get_details(sw_spec_id)

{'metadata': {'name': 'statsmodels software_spec',
  'asset_id': 'a3445242-780d-4350-8d19-ff5ac6030191',
  'href': '/v2/software_specifications/a3445242-780d-4350-8d19-ff5ac6030191',
  'asset_type': 'software_specification',
  'created_at': '2025-05-07T09:00:52Z',
  'life_cycle': {'since_version': '5.2.0'}},
 'entity': {'software_specification': {'type': 'derived',
   'display_name': 'statsmodels software_spec',
   'package_extensions': [{'metadata': {'space_id': '8a13841b-df99-4b4d-bf2a-161ad2e33980',
      'usage': {'last_updated_at': '2025-05-07T09:00:48Z',
       'last_updater_id': '1000331001',
       'last_update_time': 1746608448593,
       'last_accessed_at': '2025-05-07T09:00:48Z',
       'last_access_time': 1746608448593,
       'last_accessor_id': '1000331001',
       'access_count': 0},
      'rov': {'mode': 0,
       'collaborator_ids': {},
       'member_roles': {'1000331001': {'user_iam_id': '1000331001',
         'roles': ['OWNER']}}},
      'is_linked_with_sub_containe

#### Store the function

In [14]:
meta_props = {
    client.repository.FunctionMetaNames.NAME: "statsmodels function",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: sw_spec_id,
}

function_details = client.repository.store_function(
    meta_props=meta_props, function=deployable_callable
)
function_id = client.repository.get_function_id(function_details)

#### Get function details

In [15]:
client.repository.get_details(function_id)

{'metadata': {'name': 'statsmodels function',
  'space_id': '8a13841b-df99-4b4d-bf2a-161ad2e33980',
  'resource_key': '5d669d9c-4f17-47ff-a708-ca513ad1aaf5',
  'id': 'eb782a4f-db22-458b-9d15-9db52b5ff7f4',
  'created_at': '2025-05-07T09:17:48Z',
  'rov': {'member_roles': {'1000331001': {'user_iam_id': '1000331001',
     'roles': ['OWNER']}}},
  'owner': '1000331001'},
 'entity': {'software_spec': {'id': 'a3445242-780d-4350-8d19-ff5ac6030191'},
  'type': 'python'}}

**Note:** You can see that function is successfully stored in watsonx.ai Runtime service.

In [16]:
client.repository.list_functions()

,ID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,eb782a4f-db22-458b-9d15-9db52b5ff7f4,statsmodels function,2025-05-07T09:17:48Z,python,,


<a id="deploy"></a>
## 4. Create online deployment
You can use commands bellow to create online deployment for stored function (web service).

#### Create online deployment of a python function

In [17]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of statsmodels function",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
}

function_deployment = client.deployments.create(function_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: 'eb782a4f-db22-458b-9d15-9db52b5ff7f4' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.....
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='ad172150-7d99-410c-b899-b1357b9b0478'
-----------------------------------------------------------------------------------------------




In [18]:
client.deployments.list()

,ID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,ad172150-7d99-410c-b899-b1357b9b0478,Deployment of statsmodels function,ready,2025-05-07T09:18:21.937Z,function,,


Get deployment id.

In [19]:
deployment_id = client.deployments.get_id(function_deployment)
deployment_id

'ad172150-7d99-410c-b899-b1357b9b0478'

<a id="score"></a>
## 5. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [20]:
scoring_payload = {"input_data": [{"values": data}]}

In [21]:
predictions = client.deployments.score(deployment_id, scoring_payload)
print(data_description["predictions"][0]["values"])

                          0          1          2          3          4  \
nobs              10.000000  10.000000  10.000000  10.000000  10.000000   
missing            0.000000   0.000000   0.000000   0.000000   0.000000   
mean               0.109419  -0.396759   0.230355  -0.300086  -0.404532   
std_err            0.366493   0.353975   0.292359   0.310328   0.303524   
upper_ci           0.827732   0.297020   0.803369   0.308147   0.190363   
lower_ci          -0.608894  -1.090538  -0.342659  -0.908318  -0.999428   
std                1.158952   1.119368   0.924521   0.981344   0.959826   
iqr                1.658504   0.502628   0.783272   1.306470   1.403269   
iqr_normal         1.229451   0.372599   0.580640   0.968488   1.040245   
mad                0.914476   0.643780   0.667696   0.786667   0.797205   
mad_normal         1.146126   0.806858   0.836833   0.985941   0.999148   
coef_var          10.591883  -2.821279   4.013468  -3.270214  -2.372679   
range              3.4888

<a id="cleanup"></a>
## 6. Clean up   

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps     

You successfully completed this notebook! You learned how to use watsonx.ai for function deployment and scoring with custom `software_spec`.  

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Jan Sołtysik**, Software Engineer Intern at watsonx.ai.

**Rafał Chrzanowski**, Software Engineer Intern at watsonx.ai.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.